# Генерация из GPT

Будем генерировать твиты по примерам из аккаунта "Усы Пескова": https://twitter.com/Sandy_mustache

In [1]:
!pip install transformers

In [2]:
!wget https://gist.githubusercontent.com/avidale/d3da0ded85a4a16db6eb84d8331638ce/raw/a188084e5ef37b43b01fef0534b55c865b9a569e/tweets.txt

"wget" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [3]:
with open('tweets.txt', 'r', encoding='utf-8') as f:
    tweets = f.read().strip().split('\n\n')
print(len(tweets))
for i in range(3):
    print(tweets[i])

26
Соловьев наконец-то вышел на новый уровень - теперь его стали банить и в офлайне
Дарим мы тебе бутылку игристого вина. Пить тебе еще рано, но встретиться с ней за некоторые преступления ты уже можешь. ПОЗ-ДРАВ-ЛЯ-ЕМ!
Да. Еще очень многие помнят, что такое госплан, как планировалось, талоны на еду, очереди, дефицит, выездные визы. Но спасибо, что напомнил


In [4]:
import torch
device = torch.device("cuda")

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [6]:
model_name = 'sberbank-ai/rugpt3large_based_on_gpt2'
#model_name = 'Grossmend/rudialogpt3_medium_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
import random

In [8]:
sep = '\n***\n'
# sep = '\n27479153	Sandy_mustache	2021-02-18 16:44:00	'

prefix = sep.join([''] + random.sample(tweets, k=5) + [''])

tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}
end_token_id = tokenizer.encode('*')[0]
print(prefix)


***
Когда работа - улёт
***
Видимо, чтобы проголосовать онлайн, нужно будет прийти на участок с паспортом, зарегистрироваться в журнале участковой комиссии и удобно проголосовать не в кабинке, а на компьютере
***
Соловьев наконец-то вышел на новый уровень - теперь его стали банить и в офлайне
***
А чего они так боятся? Нам же все время говорят, что законопослушным гражданам скрывать нечего
***
В сеть утекли кадры с репетиции следующей инаугурации Путина
***



In [9]:
size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False, 
    max_length=size+128, 
    repetition_penalty=1.2, 
    temperature=0.7,
    num_beams=3,
)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Пресс-секретарь президента России Дмитрий Песков заявил, что Владимир Путин не будет баллотироваться на пост главы государства в 2018 году. Об этом сообщает РИА «Новости».

«Президент не будет баллотироваться на пост главы государства в 2018 году», — сказал Песков.

Ранее сообщалось, что Песков заявил, что Владимир Путин не будет баллотироваться на пост главы государства в 2018 году.

https://lenta.ru/news/2018/12/28/putin/

***
Надеюсь, что Путин не будет баллотироваться на пост главы государства в 2018 году

***



# Диалоги

Логика та же самая, что и с твитами. Только выбрали разделитель в стиле bash.org

In [10]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

In [11]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
for _ in range(5):
    result = respond_to_dialog(history[-10:])
    next_sentence = input(result + '\n')
    history.append(result)
    history.append(next_sentence)

In [12]:
for i in range(10):
    print(respond_to_dialog(['Давай поговорим о домашних животных', 'Каких питомцев вы любите?\n--------', 
                             'Давай поговорим о машинах', 'Какого цвета твой автомобиль?\n--------',
                             'Давай поговорим о физике']))

Что такое физика?
Что такое электричество?
Ты знаешь, что такое гравитация?
Что такое физика?
Какой цвет нравится тебе больше всего?
Что ты знаешь об электричестве?
Что такое физика?
Что это такое?
Что такое физика?
Что такое физика?
